In [2]:
# @title ## 1.1. Install Dependencies
# @markdown Clone Kohya Trainer from GitHub and check for updates. Use textbox below if you want to checkout other branch or old commit. Leave it empty to stay the HEAD on main.  This will also install the required libraries.
import os
import zipfile
import shutil
import time
from subprocess import getoutput
from IPython.utils import capture


# root_dir
root_dir = "C:\hippopenny\kohya-trainer"
pretrained_model = os.path.join(root_dir, "pretrained", "Stable-Diffusion-v1-5.safetensors")

# training_dir
project_dir = "C:\hippopenny\kohya-trainer\projects\lora1121"
training_dir = project_dir
config_dir = os.path.join(training_dir, "config")
output_dir = os.path.join(training_dir, "output")  # @param {'type':'string'}
logging_dir = os.path.join(training_dir, "logs")
train_data_dir = os.path.join(training_dir, "train_data")


# repo_dir
accelerate_config = os.path.join(root_dir, "default_config.yaml")
tools_dir = os.path.join(root_dir, "tools")
finetune_dir = os.path.join(root_dir, "finetune")



# Data Preparation


In [3]:
# @title ## 3.1. Locating Train Data Directory
# @markdown Define location of your training data. This cell will also create a folder based on your input.
# @markdown This folder will serve as the target folder for scraping, tagging, bucketing, and training in the next cell.
import os


train_data_dir = os.path.join(training_dir, "train_data")  # @param {'type' : 'string'}

os.makedirs(train_data_dir, exist_ok=True)
print(f"Your train data directory : {train_data_dir}")

Your train data directory : C:\hippopenny\sd-scripts\LoRA1120\train_data


# Data Preprocessing

In [4]:
# @title ## 4.1. Data Cleaning
import os
import random
import concurrent.futures
from tqdm import tqdm
from PIL import Image

os.chdir(root_dir)

test = os.listdir(train_data_dir)
# @markdown This section will delete unnecessary files and unsupported media such as `.mp4`, `.webm`, and `.gif`.
# @markdown Set the `convert` parameter to convert your transparent dataset with an alpha channel (RGBA) to RGB and give it a white background.
convert = False  # @param {type:"boolean"}
# @markdown You can choose to give it a `random_color` background instead of white by checking the corresponding option.
random_color = False  # @param {type:"boolean"}
# @markdown Use the `recursive` option to preprocess subfolders as well.
recursive = False #  @param {type:"boolean"}


batch_size = 32
supported_types = [
    ".png",
    ".jpg",
    ".jpeg",
    ".webp",
    ".bmp",
    ".caption",
    ".npz",
    ".txt",
    ".json",
]

background_colors = [
    (255, 255, 255),
    (0, 0, 0),
    (255, 0, 0),
    (0, 255, 0),
    (0, 0, 255),
    (255, 255, 0),
    (255, 0, 255),
    (0, 255, 255),
]

def clean_directory(directory):
    for item in os.listdir(directory):
        file_path = os.path.join(directory, item)
        if os.path.isfile(file_path):
            file_ext = os.path.splitext(item)[1]
            if file_ext not in supported_types:
                print(f"Deleting file {item} from {directory}")
                os.remove(file_path)
        elif os.path.isdir(file_path) and recursive:
            clean_directory(file_path)

def process_image(image_path):
    img = Image.open(image_path)
    img_dir, image_name = os.path.split(image_path)

    if img.mode in ("RGBA", "LA"):
        if random_color:
            background_color = random.choice(background_colors)
        else:
            background_color = (255, 255, 255)
        bg = Image.new("RGB", img.size, background_color)
        bg.paste(img, mask=img.split()[-1])

        if image_name.endswith(".webp"):
            bg = bg.convert("RGB")
            new_image_path = os.path.join(img_dir, image_name.replace(".webp", ".jpg"))
            bg.save(new_image_path, "JPEG")
            os.remove(image_path)
            print(f" Converted image: {image_name} to {os.path.basename(new_image_path)}")
        else:
            bg.save(image_path, "PNG")
            print(f" Converted image: {image_name}")
    else:
        if image_name.endswith(".webp"):
            new_image_path = os.path.join(img_dir, image_name.replace(".webp", ".jpg"))
            img.save(new_image_path, "JPEG")
            os.remove(image_path)
            print(f" Converted image: {image_name} to {os.path.basename(new_image_path)}")
        else:
            img.save(image_path, "PNG")

def find_images(directory):
    images = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".png") or file.endswith(".webp"):
                images.append(os.path.join(root, file))
    return images

clean_directory(train_data_dir)
images = find_images(train_data_dir)
num_batches = len(images) // batch_size + 1

if convert:
    with concurrent.futures.ThreadPoolExecutor() as executor:
        for i in tqdm(range(num_batches)):
            start = i * batch_size
            end = start + batch_size
            batch = images[start:end]
            executor.map(process_image, batch)

    print("All images have been converted")


## 4.2. Data Annotation
You can choose to train a model using captions. We're using [BLIP](https://huggingface.co/spaces/Salesforce/BLIP) for image captioning and [Waifu Diffusion 1.4 Tagger](https://huggingface.co/spaces/SmilingWolf/wd-v1-4-tags) for image tagging similar to Danbooru.
- Use BLIP Captioning for: `General Images`
- Use Waifu Diffusion 1.4 Tagger V2 for: `Anime and Manga-style Images`

Unlike dreambooth, you can use both for fine-tuning!


In [6]:
#@title ### 4.2.1. BLIP Captioning
#@markdown BLIP is a pre-training framework for unified vision-language understanding and generation, which achieves state-of-the-art results on a wide range of vision-language tasks. It can be used as a tool for image captioning, for example, `astronaut riding a horse in space`.
import os

os.chdir(finetune_dir)

batch_size = 8 #@param {type:'number'}
max_data_loader_n_workers = 2 #@param {type:'number'}
beam_search = True #@param {type:'boolean'}
min_length = 5 #@param {type:"slider", min:0, max:100, step:5.0}
max_length = 75 #@param {type:"slider", min:0, max:100, step:5.0}
#@markdown Use the `recursive` option to process subfolders as well.
recursive = True #@param {type:"boolean"}
#@markdown Debug while captioning, it will print your image file with generated captions.
verbose_logging = True #@param {type:"boolean"}

config = {
    "_train_data_dir" : train_data_dir,
    "batch_size" : batch_size,
    "beam_search" : beam_search,
    "min_length" : min_length,
    "max_length" : max_length,
    "debug" : verbose_logging,
    "caption_extension" : ".caption",
    "max_data_loader_n_workers" : max_data_loader_n_workers,
    "recursive" : recursive
}

args = ""
for k, v in config.items():
    if k.startswith("_"):
        args += f'"{v}" '
    elif isinstance(v, str):
        args += f'--{k}="{v}" '
    elif isinstance(v, bool) and v:
        args += f"--{k} "
    elif isinstance(v, float) and not isinstance(v, bool):
        args += f"--{k}={v} "
    elif isinstance(v, int) and not isinstance(v, bool):
        args += f"--{k}={v} "

final_args = f"python make_captions.py {args}"
print(final_args)
os.chdir(finetune_dir)
!{final_args}

python make_captions.py "C:\hippopenny\sd-scripts\LoRA1120\train_data" --batch_size=8 --beam_search --min_length=5 --max_length=75 --debug --caption_extension=".caption" --max_data_loader_n_workers=2 --recursive 
load images from C:\hippopenny\sd-scripts\LoRA1120\train_data
found 10 images.
loading BLIP caption: https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_large_caption.pth
load checkpoint from https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_large_caption.pth
BLIP loaded
C:\hippopenny\sd-scripts\LoRA1120\train_data\1_img\00106-421590727.png a woman in a white dress sitting at a table
C:\hippopenny\sd-scripts\LoRA1120\train_data\1_img\00107-421590728.png a woman with a white top sitting on a bench
C:\hippopenny\sd-scripts\LoRA1120\train_data\1_img\00108-421590729.png a woman in a white dress sitting on a bench
C:\hippopenny\sd-scripts\LoRA1120\train_data\1_img\00109-421590730.png a woman sitting at a table with a plate of fo

A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'

  0%|          | 0/2 [00:00<?, ?it/s]A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'
A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'

100%|██████████| 2/2 [00:02<00:00,  1.35s/it]


In [ ]:
#@title ### 4.2.2. Waifu Diffusion 1.4 Tagger V2
import os

os.chdir(finetune_dir)

#@markdown [Waifu Diffusion 1.4 Tagger V2](https://huggingface.co/spaces/SmilingWolf/wd-v1-4-tags) is a Danbooru-styled image classification model developed by SmilingWolf. It can also be useful for general image tagging, for example, `1girl, solo, looking_at_viewer, short_hair, bangs, simple_background`.
batch_size = 8 #@param {type:'number'}
max_data_loader_n_workers = 2 #@param {type:'number'}
model = "SmilingWolf/wd-v1-4-convnextv2-tagger-v2" #@param ["SmilingWolf/wd-v1-4-convnextv2-tagger-v2", "SmilingWolf/wd-v1-4-swinv2-tagger-v2", "SmilingWolf/wd-v1-4-convnext-tagger-v2", "SmilingWolf/wd-v1-4-vit-tagger-v2"]
#@markdown Use the `recursive` option to process subfolders as well
recursive = False #@param {type:"boolean"}
#@markdown Debug while tagging, it will print your image file with general tags and character tags.
verbose_logging = True #@param {type:"boolean"}
#@markdown Separate `undesired_tags` with comma `(,)` if you want to remove multiple tags, e.g. `1girl,solo,smile`.
undesired_tags = "" #@param {type:'string'}
#@markdown  Adjust `general_threshold` for pruning tags (less tags, less flexible). `character_threshold` is useful if you want to train with character tags, e.g. `hakurei reimu`.
general_threshold = 0.35 #@param {type:"slider", min:0, max:1, step:0.05}
character_threshold = 0.35 #@param {type:"slider", min:0, max:1, step:0.05}

config = {
    "_train_data_dir": train_data_dir,
    "batch_size": batch_size,
    "repo_id": model,
    "recursive": recursive,
    "remove_underscore": True,
    "general_threshold": general_threshold,
    "character_threshold": character_threshold,
    "caption_extension": ".txt",
    "max_data_loader_n_workers": max_data_loader_n_workers,
    "debug": verbose_logging,
    "undesired_tags": undesired_tags
}

args = ""
for k, v in config.items():
    if k.startswith("_"):
        args += f'"{v}" '
    elif isinstance(v, str):
        args += f'--{k}="{v}" '
    elif isinstance(v, bool) and v:
        args += f"--{k} "
    elif isinstance(v, float) and not isinstance(v, bool):
        args += f"--{k}={v} "
    elif isinstance(v, int) and not isinstance(v, bool):
        args += f"--{k}={v} "

final_args = f"python tag_images_by_wd14_tagger.py {args}"

os.chdir(finetune_dir)
!{final_args}

In [24]:
# @title ### 4.2.3. Custom Caption/Tag
import os

os.chdir(root_dir)

# @markdown Add or remove custom tags here. You can refer to this [cheatsheet](https://rentry.org/kohyaminiguide#c-custom-tagscaption) for more information.
extension = ".caption"  # @param [".txt", ".caption"]
custom_tag = "hppg"  # @param {type:"string"}
#@markdown Use the `recursive` option to process subfolders as well
# @markdown > Specify `--all` to process all subfolders/`recursive`
sub_folder = "" #@param {type: "string"}
# @markdown Enable this to append custom tags at the end of lines.
append = False  # @param {type:"boolean"}
# @markdown Enable this if you want to remove captions/tags instead.
remove_tag = False  # @param {type:"boolean"}
recursive = True

if sub_folder == "":
    image_dir = train_data_dir
elif sub_folder == "--all":
    image_dir = train_data_dir
    recursive = True
elif sub_folder.startswith("/content"):
    image_dir = sub_folder
else:
    image_dir = os.path.join(train_data_dir, sub_folder)
    os.makedirs(image_dir, exist_ok=True)

def read_file(filename):
    with open(filename, "r") as f:
        contents = f.read()
    return contents

def write_file(filename, contents):
    with open(filename, "w") as f:
        f.write(contents)

def process_tags(filename, custom_tag, append, remove_tag):
    contents = read_file(filename)
    tags = [tag.strip() for tag in contents.split(',')]
    custom_tags = [tag.strip() for tag in custom_tag.split(',')]

    for custom_tag in custom_tags:
        custom_tag = custom_tag.replace("_", " ")
        if remove_tag:
            i = 0
            while i < len(tags):
                tags[i] = tags[i].replace(custom_tag, '')
                print('custom tag', tags[i])
                i += 1
            i = 0
        else:
            if custom_tag not in tags:
                if append:
                    tags.append(custom_tag)
                else:
                    tags.insert(0, custom_tag)

    contents = ', '.join(tags)
    write_file(filename, contents)

def process_directory(image_dir, tag, append, remove_tag, recursive):
    for filename in os.listdir(image_dir):
        file_path = os.path.join(image_dir, filename)

        if os.path.isdir(file_path) and recursive:
            process_directory(file_path, tag, append, remove_tag, recursive)
        elif filename.endswith(extension):
            process_tags(file_path, tag, append, remove_tag)

tag = custom_tag
if not any(
    [filename.endswith(extension) for filename in os.listdir(image_dir)]
):
    for filename in os.listdir(image_dir):
        if filename.endswith((".png", ".jpg", ".jpeg", ".webp", ".bmp")):
            open(
                os.path.join(image_dir, filename.split(".")[0] + extension),
                "w",
            ).close()

if custom_tag:
    process_directory(image_dir, tag, append, remove_tag, recursive)


In [5]:
# #@title ## 4.3. Create Metadata File
# import os

# os.chdir(finetune_dir)

# # @markdown Merge tags and/or captions exist in `train_data_dir` into one metadata JSON file, which will be used as the input for the bucketing section.
# metadata = os.path.join(root_dir,"train_data/meta_clean.json") #@param {type:"string"}
# # @markdown Use `recursive` option to process subfolders as well
# recursive = False #@param {type:"boolean"}
# # @markdown Use `clean_caption` option to clean such as duplicate tags, `women` to `girl`, etc
# clean_caption = False #@param {type:"boolean"}

# config = {
#     "_train_data_dir": train_data_dir,
#     "_out_json": metadata,
#     "recursive": recursive,
#     "full_path": recursive,
#     "clean_caption": clean_caption
# }

# args = ""
# for k, v in config.items():
#     if k.startswith("_"):
#         args += f'"{v}" '
#     elif isinstance(v, str):
#         args += f'--{k}="{v}" '
#     elif isinstance(v, bool) and v:
#         args += f"--{k} "
#     elif isinstance(v, float) and not isinstance(v, bool):
#         args += f"--{k}={v} "
#     elif isinstance(v, int) and not isinstance(v, bool):
#         args += f"--{k}={v} "

# os.chdir(finetune_dir)
# final_args = f"python merge_all_to_metadata.py {args}"
# !{final_args}


In [25]:
# # @title ## 4.4. Bucketing and Latents Caching

# # What happens here is that instead of doing VAE image encoding again and again for every batch, epochs, we preprocess that once and save latents
# # output: metadata.lat including image dims; latens are saved with same file name to train_data folder with .npz extension 
# # @markdown This code will create buckets based on the `max_resolution` provided for multi-aspect ratio training, and then convert all images within the `train_data_dir` to latents.
# v2 = False  # @param{type:"boolean"}
# # VAE model
# model_dir = "C:\hippopenny\stable-diffusion-webui\models\VAE\\vae-ft-mse-840000-ema-pruned.ckpt"  # @param {'type' : 'string'}
# input_json = metadata  # @param {'type' : 'string'}
# output_json = os.path.join(root_dir,"train_data/meta_lat.json")  # @param {'type' : 'string'}
# batch_size = 8  # @param {'type':'integer'}
# max_data_loader_n_workers = 2  # @param {'type':'integer'}
# max_resolution = "512,512"  # @param ["512,512", "640,640", "768,768"] {allow-input: false}
# mixed_precision = "no"  # @param ["no", "fp16", "bf16"] {allow-input: false}
# flip_aug = False  # @param{type:"boolean"}
# #@markdown Use the `recursive` option to process subfolders as well
# recursive = False #@param {type:"boolean"}

# config = {
#     "_train_data_dir": train_data_dir,
#     "_in_json": input_json,
#     "_out_json": output_json,
#     "_model_name_or_path": model_dir,
#     "recursive": recursive,
#     "full_path": recursive,
#     "v2": v2,
#     "flip_aug": flip_aug,
#     "min_bucket_reso": 320 if max_resolution != "512,512" else 256,
#     "max_bucket_reso": 1280 if max_resolution != "512,512" else 1024,
#     "batch_size": batch_size,
#     "max_data_loader_n_workers": max_data_loader_n_workers,
#     "max_resolution": max_resolution,
#     "mixed_precision": mixed_precision,
# }

# args = ""
# for k, v in config.items():
#     if k.startswith("_"):
#         args += f'"{v}" '
#     elif isinstance(v, str):
#         args += f'--{k}="{v}" '
#     elif isinstance(v, bool) and v:
#         args += f"--{k} "
#     elif isinstance(v, float) and not isinstance(v, bool):
#         args += f"--{k}={v} "
#     elif isinstance(v, int) and not isinstance(v, bool):
#         args += f"--{k}={v} "

# os.chdir(finetune_dir)
# final_args = f"python prepare_buckets_latents.py {args}"
# !{final_args}

# V. Training model


In [28]:
# @title ## 5.1. Model Config

v2 = False  # @param {type:"boolean"}
v_parameterization = False  # @param {type:"boolean"}
project_name = "hppg"  # @param {type:"string"}

if not project_name:
    project_name = "last"

sample_dir = os.path.join(output_dir, "sample")
for dir in [output_dir, sample_dir]:
    os.makedirs(dir, exist_ok=True)

print("Project Name: ", project_name)
print("Model Version: Stable Diffusion V1.x") if not v2 else ""
print("Model Version: Stable Diffusion V2.x") if v2 and not v_parameterization else ""
print("Model Version: Stable Diffusion V2.x 768v") if v2 and v_parameterization else ""
print(
    "Pretrained Model Path: ", pretrained_model
) if pretrained_model else print("No Pretrained Model path specified.")
print("Output Path: ", output_dir)

Project Name:  hppg
Model Version: Stable Diffusion V1.x
Pretrained Model Path:  C:\hippopenny\sd-scripts\pretrained\stable_diffusion_1_5-pruned.safetensors
Output Path:  C:\hippopenny\sd-scripts\LoRA1120\output


In [36]:


os.chdir(root_dir)

accelerate_conf = {
    "config_file" : accelerate_config,
    "num_cpu_threads_per_process" : 8,
}

train_conf = {
    "sample_prompts" : os.path.join(config_dir, "sample_prompt.txt"),
    "pretrained_model_name_or_path": pretrained_model,
    "train_data_dir": train_data_dir,
    "output_name": "hppg",
    "output_dir": output_dir,
    "log_prefix": "hppg",
    "logging_dir": logging_dir,
    "sample_every_n_steps": 1,
    "max_train_epochs": 50,
    "train_batch_size": 5,
    "save_n_epoch_ratio": 3,
    "caption_extension": "caption",
    "prior_loss_weight": 1.0,
    "mixed_precision": "fp16",
    "save_precision": "fp16",
    "save_model_as": "safetensors",
    "resolution": 512,
    "enable_bucket": True,
    "min_bucket_reso": 256,
    "max_bucket_reso": 1024,
    "cache_latents": True,
    "max_token_length": 225,
    "use_8bit_adam": True,
    "gradient_accumulation_steps": 1,
    "clip_skip": 1,
    "network_dim": 32,
    "network_alpha": 32,
    "network_module": "networks.lora",
    "learning_rate": 0.0001,
    "text_encoder_lr": 5e-05,
    "training_comment":"this_comment_will_be_stored_in_the_metadata",
    "lr_scheduler": "constant",
    "shuffle_caption": True,
    "xformers": True
}

def train(config):
    args = ""
    for k, v in config.items():
        if k.startswith("_"):
            args += f'"{v}" '
        elif isinstance(v, str):
            args += f'--{k}="{v}" '
        elif isinstance(v, bool) and v:
            args += f"--{k} "
        elif isinstance(v, float) and not isinstance(v, bool):
            args += f"--{k}={v} "
        elif isinstance(v, int) and not isinstance(v, bool):
            args += f"--{k}={v} "

    return args

accelerate_args = train(accelerate_conf)
train_args = train(train_conf)
final_args = f"accelerate launch {accelerate_args} train_network.py {train_args}"
print(final_args)
!{final_args}

accelerate launch --config_file="C:\hippopenny\sd-scripts\default_config.yaml" --num_cpu_threads_per_process=8  train_network.py --sample_prompts="C:\hippopenny\sd-scripts\LoRA1120\config\sample_prompt.txt" --pretrained_model_name_or_path="C:\hippopenny\sd-scripts\pretrained\stable_diffusion_1_5-pruned.safetensors" --train_data_dir="C:\hippopenny\sd-scripts\LoRA1120\train_data" --output_name="hppg" --output_dir="C:\hippopenny\sd-scripts\LoRA1120\output" --log_prefix="hppg" --logging_dir="C:\hippopenny\sd-scripts\LoRA1120\logs" --sample_every_n_steps=1 --max_train_epochs=50 --train_batch_size=5 --save_n_epoch_ratio=3 --caption_extension="caption" --prior_loss_weight=1.0 --mixed_precision="fp16" --save_precision="fp16" --save_model_as="safetensors" --resolution=512 --enable_bucket --min_bucket_reso=256 --max_bucket_reso=1024 --cache_latents --max_token_length=225 --use_8bit_adam --gradient_accumulation_steps=1 --clip_skip=1 --network_dim=32 --network_alpha=32 --network_module="networks.l

In [5]:

# another way to train
sample_prompt = os.path.join(config_dir, "sample_prompt.txt")

accelerate_conf = {
    "config_file" : accelerate_config,
    "num_cpu_threads_per_process" : 1,
}

dataset_repeats = 10

train_conf = {
    "sample_prompts" : sample_prompt,
    "pretrained_model_name_or_path": pretrained_model,
    "train_data_dir": train_data_dir,
    "output_name": "hppg",
    "output_dir": output_dir,
    "log_prefix": "hppg",
    "logging_dir": logging_dir,
    "sample_every_n_steps": 10,
    "max_train_epochs": 50,
    "train_batch_size": 5,
    "save_n_epoch_ratio": 3,
    "caption_extension": "caption",
    "prior_loss_weight": 1.0,
    "mixed_precision": "fp16",
    "save_precision": "fp16",
    "save_model_as": "safetensors",
    "resolution": 512,
    "enable_bucket": True,
    "min_bucket_reso": 256,
    "max_bucket_reso": 1024,
    "cache_latents": True,
    "max_token_length": 225,
    "use_8bit_adam": True,
    "gradient_accumulation_steps": 1,
    "clip_skip": 1,
    "network_dim": 32,
    "network_alpha": 32,
    "network_module": "networks.lora",
    "learning_rate": 0.0001,
    "text_encoder_lr": 5e-05,
    "training_comment":"this_comment_will_be_stored_in_the_metadata",
    "lr_scheduler": "constant",
    "shuffle_caption": True,
    "xformers": True
}

def train(config):
    args = ""
    for k, v in config.items():
        if k.startswith("_"):
            args += f'"{v}" '
        elif isinstance(v, str):
            args += f'--{k}="{v}" '
        elif isinstance(v, bool) and v:
            args += f"--{k} "
        elif isinstance(v, float) and not isinstance(v, bool):
            args += f"--{k}={v} "
        elif isinstance(v, int) and not isinstance(v, bool):
            args += f"--{k}={v} "

    return args


accelerate_args = train(accelerate_conf)
train_args = train(train_conf)
final_args = f"accelerate launch {accelerate_args} train_network.py {train_args}"

os.chdir(root_dir)
!{final_args}




^C


# VI. Testing

In [9]:
# @title ## 6.1. Visualize loss graph (Optional)
# training_logs_path = "/content/LoRA/logs"  # @param {type : "string"}

os.chdir(root_dir)
%load_ext tensorboard
%tensorboard --logdir {logging_dir}

In [8]:
logging_dir

'C:\\hippopenny\\sd-scripts\\LoRA1120\\logs'

In [17]:
# @title ## 6.2. Interrogating LoRA Weights
# @markdown Now you can check if your LoRA trained properly.
import os
import torch
import json
from safetensors.torch import load_file
from safetensors.torch import safe_open

# @markdown If you used `clip_skip = 2` during training, the values of `lora_te_text_model_encoder_layers_11_*` will be `0.0`, this is normal. These layers are not trained at this value of `Clip Skip`.
network_weight = ""  # @param {'type':'string'}
verbose = False  # @param {type:"boolean"}

def is_safetensors(path):
    return os.path.splitext(path)[1].lower() == ".safetensors"

def load_weight_data(file_path):
    if is_safetensors(file_path):
        return load_file(file_path)
    else:
        return torch.load(file_path, map_location="cuda")

def extract_lora_weights(weight_data):
    lora_weights = [
        (key, weight_data[key])
        for key in weight_data.keys()
        if "lora_up" in key or "lora_down" in key
    ]
    return lora_weights

def print_lora_weight_stats(lora_weights):
    print(f"Number of LoRA modules: {len(lora_weights)}")

    for key, value in lora_weights:
        value = value.to(torch.float32)
        print(f"{key}, {torch.mean(torch.abs(value))}, {torch.min(torch.abs(value))}")

def print_metadata(file_path):
    if is_safetensors(file_path):
        with safe_open(file_path, framework="pt") as f:
            metadata = f.metadata()
        if metadata is not None:
            print(f"\nLoad metadata for: {file_path}")
            print(json.dumps(metadata, indent=4))
    else:
        print("No metadata saved, your model is not in safetensors format")

def main(file_path, verbose: bool):
    weight_data = load_weight_data(file_path)

    if verbose:
        lora_weights = extract_lora_weights(weight_data)
        print_lora_weight_stats(lora_weights)

    print_metadata(file_path)

if __name__ == "__main__":
    main(network_weight, verbose)

FileNotFoundError: [Errno 2] No such file or directory: ''

In [ ]:
# @title ## 6.3. Inference
%store -r

# @markdown ### LoRA Config
# @markdown Currently, `LoHa` and `LoCon_Lycoris` are not supported. Please run `Portable Web UI` instead
network_weight = ""  # @param {'type':'string'}
network_mul = 0.7  # @param {type:"slider", min:-1, max:2, step:0.05}
network_module = "networks.lora"
network_args = ""

# @markdown ### <br> General Config
v2 = False  # @param {type:"boolean"}
v_parameterization = False  # @param {type:"boolean"}
prompt = "masterpiece, best quality, 1girl, aqua eyes, baseball cap, blonde hair, closed mouth, earrings, green background, hat, hoop earrings, jewelry, looking at viewer, shirt, short hair, simple background, solo, upper body, yellow shirt"  # @param {type: "string"}
negative = "lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry"  # @param {type: "string"}
model = "/content/pretrained_model/AnyLoRA.safetensors"  # @param {type: "string"}
vae = ""  # @param {type: "string"}
outdir = "/content/tmp"  # @param {type: "string"}
scale = 7  # @param {type: "slider", min: 1, max: 40}
sampler = "ddim"  # @param ["ddim", "pndm", "lms", "euler", "euler_a", "heun", "dpm_2", "dpm_2_a", "dpmsolver","dpmsolver++", "dpmsingle", "k_lms", "k_euler", "k_euler_a", "k_dpm_2", "k_dpm_2_a"]
steps = 28  # @param {type: "slider", min: 1, max: 100}
precision = "fp16"  # @param ["fp16", "bf16"] {allow-input: false}
width = 512  # @param {type: "integer"}
height = 768  # @param {type: "integer"}
images_per_prompt = 4  # @param {type: "integer"}
batch_size = 4  # @param {type: "integer"}
clip_skip = 2  # @param {type: "slider", min: 1, max: 40}
seed = -1  # @param {type: "integer"}

final_prompt = f"{prompt} --n {negative}"

config = {
    "v2": v2,
    "v_parameterization": v_parameterization,
    "network_module": network_module,
    "network_weight": network_weight,
    "network_mul": float(network_mul),
    "network_args": eval(network_args) if network_args else None,
    "ckpt": model,
    "outdir": outdir,
    "xformers": True,
    "vae": vae if vae else None,
    "fp16": True,
    "W": width,
    "H": height,
    "seed": seed if seed > 0 else None,
    "scale": scale,
    "sampler": sampler,
    "steps": steps,
    "max_embeddings_multiples": 3,
    "batch_size": batch_size,
    "images_per_prompt": images_per_prompt,
    "clip_skip": clip_skip if not v2 else None,
    "prompt": final_prompt,
}

args = ""
for k, v in config.items():
    if k.startswith("_"):
        args += f'"{v}" '
    elif isinstance(v, str):
        args += f'--{k}="{v}" '
    elif isinstance(v, bool) and v:
        args += f"--{k} "
    elif isinstance(v, float) and not isinstance(v, bool):
        args += f"--{k}={v} "
    elif isinstance(v, int) and not isinstance(v, bool):
        args += f"--{k}={v} "

final_args = f"python gen_img_diffusers.py {args}"

os.chdir(repo_dir)
!{final_args}

In [ ]:
#@title ## 6.4. Launch Portable Web UI
import os
import random
import shutil
import zipfile
import time
import json
from google.colab import drive
from datetime import timedelta
from subprocess import getoutput
from IPython.display import clear_output, display, HTML
from IPython.utils import capture
from tqdm import tqdm

webui_dir = os.path.join(root_dir, "stable-diffusion-webui")
tmp_dir = os.path.join(root_dir, "tmp")
patches_dir = os.path.join(root_dir, "patches")
deps_dir = os.path.join(root_dir, "deps")
extensions_dir = os.path.join(webui_dir, "extensions")
control_dir = os.path.join(webui_dir, "models/ControlNet")

webui_models_dir = os.path.join(webui_dir, "models/Stable-diffusion")
webui_lora_dir = os.path.join(webui_dir, "models/Lora")
webui_vaes_dir = os.path.join(webui_dir, "models/VAE")

control_net_max_models_num = 2
theme = "ogxBGreen"

default_prompt = "masterpiece, best quality,"
default_neg_prompt = "(worst quality, low quality:1.4)"
default_sampler = "DPM++ 2M Karras"
default_steps = 20
default_width = 512
default_height = 768
default_denoising_strength = 0.55
default_cfg_scale = 7

config_file = os.path.join(webui_dir, "config.json")
ui_config_file = os.path.join(webui_dir, "ui-config.json")
webui_style_path = os.path.join(webui_dir, "style.css")

os.chdir(root_dir)

for dir in [patches_dir, deps_dir]:
    os.makedirs(dir, exist_ok=True)

package_url = [
    f"https://huggingface.co/Linaqruf/fast-repo/resolve/main/anapnoe-webui.tar.lz4",
    f"https://huggingface.co/Linaqruf/fast-repo/resolve/main/anapnoe-webui-deps.tar.lz4",
    f"https://huggingface.co/Linaqruf/fast-repo/resolve/main/anapnoe-webui-cache.tar.lz4",
]

def pre_download(desc):
    for package in tqdm(package_url, desc=desc):
        with capture.capture_output() as cap:
            package_name = os.path.basename(package)
            !aria2c --console-log-level=error --summary-interval=10 -c -x 16 -k 1M -s 16 -d {root_dir} -o {package_name} {package}
            if package_name == f"anapnoe-webui-deps.tar.lz4":
                !tar -xI lz4 -f {package_name} --overwrite-dir --directory=/usr/local/lib/python3.10/dist-packages/
            else:
                !tar -xI lz4 -f {package_name} --directory=/
            os.remove(package_name)
            del cap

    if os.path.exists("/usr/local/lib/python3.10/dist-packages/ffmpy-0.3.0.dist-info"):
        shutil.rmtree("/usr/local/lib/python3.10/dist-packages/ffmpy-0.3.0.dist-info")

    s = getoutput("nvidia-smi")
    with capture.capture_output() as cap:
        if not "T4" in s:
            !pip uninstall -y xformers
            !pip install -q xformers==0.0.18 triton
        del cap


def read_config(filename):
    if filename.endswith(".json"):
        with open(filename, "r") as f:
          config = json.load(f)
    else:
        with open(filename, 'r') as f:
          config = f.read()
    return config


def write_config(filename, config):
    if filename.endswith(".json"):
        with open(filename, "w") as f:
            json.dump(config, f, indent=4)
    else:
        with open(filename, 'w', encoding="utf-8") as f:
            f.write(config)


def open_theme(filename):
    themes_folder = os.path.join(webui_dir, "extensions-builtin/sd_theme_editor/themes")
    themes_file = os.path.join(themes_folder, f"{filename}.css")
    webui_style_path = os.path.join(webui_dir, "style.css")

    style_config = read_config(webui_style_path)
    style_css_contents = style_config.split("/*BREAKPOINT_CSS_CONTENT*/")[1]

    theme_config = read_config(themes_file)
    style_data = ":host{" + theme_config + "}" + "/*BREAKPOINT_CSS_CONTENT*/" + style_css_contents
    write_config(webui_style_path, style_data)


def change_config(filename):
    config = read_config(filename)
    if not "stable-diffusion-webui" in config["disabled_extensions"]:
        config["disabled_extensions"].append("stable-diffusion-webui")
    config["outdir_txt2img_samples"] = os.path.join(tmp_dir, "outputs/txt2img-images")
    config["outdir_img2img_samples"] = os.path.join(tmp_dir, "outputs/img2img-images")
    config["outdir_extras_samples"] = os.path.join(tmp_dir, "outputs/extras-images")
    config["outdir_txt2img_grids"] = os.path.join(tmp_dir, "outputs/txt2img-grids")
    config["outdir_img2img_grids"] = os.path.join(tmp_dir, "outputs/img2img-grids")
    config["outdir_save"] = os.path.join(tmp_dir, "log/images")
    config["control_net_max_models_num"] = control_net_max_models_num
    config["control_net_models_path"] = control_dir
    config["control_net_allow_script_control"] = True
    config["additional_networks_extra_lora_path"] = webui_lora_dir
    config["CLIP_stop_at_last_layers"] = 2
    config["eta_noise_seed_delta"] = 0
    config["show_progress_every_n_steps"] = 10
    config["show_progressbar"] = True
    config["quicksettings"] = "sd_model_checkpoint, sd_vae, CLIP_stop_at_last_layers, use_old_karras_scheduler_sigmas, always_discard_next_to_last_sigma"
    write_config(filename, config)


def change_ui_config(filename):
    config = read_config(filename)
    config["txt2img/Prompt/value"] = default_prompt
    config["txt2img/Negative prompt/value"] = default_neg_prompt
    config["txt2img/Sampling method/value"] = default_sampler
    config["txt2img/Sampling steps/value"] = default_steps
    config["txt2img/Width/value"] = default_width
    config["txt2img/Height/value"] = default_height
    config["txt2img/Upscaler/value"] = "Latent (nearest-exact)"
    config["txt2img/Denoising strength/value"] = default_denoising_strength
    config["txt2img/CFG Scale/value"] = default_cfg_scale
    config["img2img/Prompt/value"] = default_prompt
    config["img2img/Negative prompt/value"] = default_neg_prompt
    config["img2img/Sampling method/value"] = default_sampler
    config["img2img/Sampling steps/value"] = default_steps
    config["img2img/Width/value"] = default_width
    config["img2img/Height/value"] = default_height
    config["img2img/Denoising strength/value"] = default_denoising_strength
    config["img2img/CFG Scale/value"] = default_cfg_scale
    write_config(filename, config)


def update_extensions():
    start_time = time.time()
    extensions_updated = []
    with tqdm(
        total=len(os.listdir(extensions_dir)),
        desc="[1;32mUpdating extensions",
        mininterval=0,
    ) as pbar:
        for dir in os.listdir(extensions_dir):
            if os.path.isdir(os.path.join(extensions_dir, dir)):
                os.chdir(os.path.join(extensions_dir, dir))
                try:
                    with capture.capture_output() as cap:
                        !git fetch origin
                        !git pull
                except Exception as e:
                    print(f"[1;32mAn error occurred while updating {dir}: {e}")

                output = cap.stdout.strip()
                if "Already up to date." not in output:
                    extensions_updated.append(dir)
                pbar.update(1)

    print("\n")
    for ext in extensions_updated:
        print(f"[1;32m- {ext} updated to new version")

    end_time = time.time()
    elapsed_time = int(end_time - start_time)

    if elapsed_time < 60:
        print(f"\n[1;32mAll extensions are up to date. Took {elapsed_time} sec")
    else:
        mins, secs = divmod(elapsed_time, 60)
        print(f"\n[1;32mAll extensions are up to date. Took {mins} mins {secs} sec")


def main():
    start_time = time.time()

    print("[1;32mInstalling...\n")

    if not os.path.exists(webui_dir):
        desc = "[1;32mUnpacking Webui"
        pre_download(desc)
    else:
        print("[1;32mAlready installed, skipping...")

    with capture.capture_output() as cap:
        os.chdir(os.path.join(webui_dir, "repositories/stable-diffusion-stability-ai"))
        !git apply {patches_dir}/stablediffusion-lowram.patch

        !sed -i "s@os.path.splitext(checkpoint_.*@os.path.splitext(checkpoint_file); map_location='cuda'@" {webui_dir}/modules/sd_models.py
        !sed -i 's@ui.create_ui().*@ui.create_ui();shared.demo.queue(concurrency_count=999999,status_update_rate=0.1)@' {webui_dir}/webui.py

        !sed -i "s@'cpu'@'cuda'@" {webui_dir}/modules/extras.py
        del cap

    end_time = time.time()
    elapsed_time = int(end_time - start_time)

    change_config(config_file)
    change_ui_config(ui_config_file)
    open_theme(theme)

    if elapsed_time < 60:
        print(f"[1;32mFinished unpacking. Took {elapsed_time} sec")
    else:
        mins, secs = divmod(elapsed_time, 60)
        print(f"[1;32mFinished unpacking. Took {mins} mins {secs} sec")

    update_extensions()

    #@markdown > Get <b>your</b> `ngrok_token` [here](https://dashboard.ngrok.com/get-started/your-authtoken)
    ngrok_token = "" #@param {type: 'string'}
    ngrok_region = "ap" #@param ["us", "eu", "au", "ap", "sa", "jp", "in"]

    with capture.capture_output() as cap:
      for file in os.listdir(output_dir):
        file_path = os.path.join(output_dir, file)
        if file_path.endswith((".safetensors", ".pt", ".ckpt")):
          !ln "{file_path}" {webui_lora_dir}

      for file in os.listdir(pretrained_model):
        file_path = os.path.join(pretrained_model, file)
        if file_path.endswith((".safetensors", ".ckpt")):
          !ln "{file_path}" {webui_models_dir}

      for file in os.listdir(vae_dir):
        file_path = os.path.join(vae_dir, file)
        if file_path.endswith(".vae.pt"):
          !ln "{file_path}" {webui_vaes_dir}

      del cap

    os.chdir(webui_dir)

    print("[1;32m")

    config = {
        "enable-insecure-extension-access": True,
        "disable-safe-unpickle": True,
        "multiple": True if not ngrok_token else False,
        "ckpt-dir": webui_models_dir,
        "vae-dir": webui_vaes_dir,
        "share": True if not ngrok_token else False,
        "no-half-vae": True,
        "lowram": True,
        "gradio-queue": True,
        "no-hashing": True,
        "disable-console-progressbars": True,
        "ngrok": ngrok_token if ngrok_token else None,
        "ngrok-region": ngrok_region if ngrok_token else None,
        "xformers": True,
        "opt-sub-quad-attention": True,
        "opt-channelslast": True,
        "theme": "dark"
    }

    args = ""
    for k, v in config.items():
        if k.startswith("_"):
            args += f'"{v}" '
        elif isinstance(v, str):
            args += f'--{k}="{v}" '
        elif isinstance(v, bool) and v:
            args += f"--{k} "
        elif isinstance(v, float) and not isinstance(v, bool):
            args += f"--{k}={v} "
        elif isinstance(v, int) and not isinstance(v, bool):
            args += f"--{k}={v} "

    final_args = f"python launch.py {args}"

    os.chdir(webui_dir)
    !{final_args}

main()

# VII. Deployment

In [ ]:
# @title ## 7.1. Upload Config
from huggingface_hub import login
from huggingface_hub import HfApi
from huggingface_hub.utils import validate_repo_id, HfHubHTTPError

# @markdown Login to Huggingface Hub
# @markdown > Get **your** huggingface `WRITE` token [here](https://huggingface.co/settings/tokens)
write_token = ""  # @param {type:"string"}
# @markdown Fill this if you want to upload to your organization, or just leave it empty.
orgs_name = ""  # @param{type:"string"}
# @markdown If your model/dataset repo does not exist, it will automatically create it.
model_name = "your-model-name"  # @param{type:"string"}
dataset_name = "your-dataset-name"  # @param{type:"string"}
make_private = False  # @param{type:"boolean"}

def authenticate(write_token):
    login(write_token, add_to_git_credential=True)
    api = HfApi()
    return api.whoami(write_token), api


def create_repo(api, user, orgs_name, repo_name, repo_type, make_private=False):
    global model_repo
    global datasets_repo

    if orgs_name == "":
        repo_id = user["name"] + "/" + repo_name.strip()
    else:
        repo_id = orgs_name + "/" + repo_name.strip()

    try:
        validate_repo_id(repo_id)
        api.create_repo(repo_id=repo_id, repo_type=repo_type, private=make_private)
        print(f"{repo_type.capitalize()} repo '{repo_id}' didn't exist, creating repo")
    except HfHubHTTPError as e:
        print(f"{repo_type.capitalize()} repo '{repo_id}' exists, skipping create repo")

    if repo_type == "model":
        model_repo = repo_id
        print(f"{repo_type.capitalize()} repo '{repo_id}' link: https://huggingface.co/{repo_id}\n")
    else:
        datasets_repo = repo_id
        print(f"{repo_type.capitalize()} repo '{repo_id}' link: https://huggingface.co/datasets/{repo_id}\n")

user, api = authenticate(write_token)

if model_name:
    create_repo(api, user, orgs_name, model_name, "model", make_private)
if dataset_name:
    create_repo(api, user, orgs_name, dataset_name, "dataset", make_private)


## 7.2. Upload with Huggingface Hub

In [ ]:
# @title ### 7.2.1. Upload LoRA
from huggingface_hub import HfApi
from pathlib import Path

%store -r

api = HfApi()

# @markdown This will be uploaded to model repo
model_path = "/content/LoRA/output"  # @param {type :"string"}
path_in_repo = ""  # @param {type :"string"}

# @markdown Now you can save your config file for future use
config_path = "/content/LoRA/config"  # @param {type :"string"}

# @markdown Other Information
commit_message = ""  # @param {type :"string"}

if not commit_message:
    commit_message = f"feat: upload {project_name} lora model"

def upload_to_hf(model_path, is_folder, is_config):
    path_obj = Path(model_path)
    trained_model = path_obj.parts[-1]

    if path_in_repo:
        trained_model = path_in_repo

    if is_config:
        trained_model = f"{project_name}_config"

    print(f"Uploading {trained_model} to https://huggingface.co/{model_repo}")
    print("Please wait...")

    if is_folder:
        api.upload_folder(
            folder_path=model_path,
            path_in_repo=trained_model,
            repo_id=model_repo,
            commit_message=commit_message,
            ignore_patterns=".ipynb_checkpoints",
        )
        print(f"Upload success, located at https://huggingface.co/{model_repo}/tree/main\n")
    else:
        api.upload_file(
            path_or_fileobj=model_path,
            path_in_repo=trained_model,
            repo_id=model_repo,
            commit_message=commit_message,
        )
        print(f"Upload success, located at https://huggingface.co/{model_repo}/blob/main/{trained_model}\n")

def upload():
    is_model_file = model_path.endswith((".ckpt", ".safetensors", ".pt"))
    upload_to_hf(model_path, not is_model_file, False)

    if config_path:
        upload_to_hf(config_path, True, True)

upload()


In [ ]:
# @title ### 7.2.2. Upload Dataset
from huggingface_hub import HfApi
from pathlib import Path
import shutil
import zipfile
import os

api = HfApi()

# @markdown This will be compressed to zip and  uploaded to datasets repo, leave it empty if not necessary
train_data_path = "/content/LoRA/train_data"  # @param {type :"string"}

# @markdown `Nerd stuff, only if you want to save training logs`
logs_path = "/content/LoRA/logs"  # @param {type :"string"}

tmp_dataset = f"/content/LoRA/{project_name}_dataset" if project_name else "/content/LoRA/tmp_dataset"
tmp_train_data = f"{tmp_dataset}/train_data"
dataset_zip = f"{tmp_dataset}.zip"

# @markdown Other Information
commit_message = ""  # @param {type :"string"}

if not commit_message:
    commit_message = f"feat: upload {project_name} dataset and logs"

os.makedirs(tmp_dataset, exist_ok=True)
os.makedirs(tmp_train_data, exist_ok=True)

def upload_dataset(dataset_path, is_zip):
    path_obj = Path(dataset_path)
    dataset_name = path_obj.parts[-1]

    print(f"Uploading {dataset_name} to https://huggingface.co/datasets/{datasets_repo}")
    print("Please wait...")

    if is_zip:
        api.upload_file(
            path_or_fileobj=dataset_path,
            path_in_repo=dataset_name,
            repo_id=datasets_repo,
            repo_type="dataset",
            commit_message=commit_message,
        )
        print(f"Upload success, located at https://huggingface.co/datasets/{datasets_repo}/blob/main/{dataset_name}\n")
    else:
        api.upload_folder(
            folder_path=dataset_path,
            path_in_repo=dataset_name,
            repo_id=datasets_repo,
            repo_type="dataset",
            commit_message=commit_message,
            ignore_patterns=".ipynb_checkpoints",
        )
        print(f"Upload success, located at https://huggingface.co/datasets/{datasets_repo}/tree/main/{dataset_name}\n")

def zip_file(folder_path):
    zip_path = f"{folder_path}.zip"
    with zipfile.ZipFile(zip_path, "w") as zip_file:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zip_file.write(os.path.join(root, file))

def move(src_path, dst_path, move_metadata):
    metadata_files = [
        "meta_cap.json",
        "meta_cap_dd.json",
        "meta_lat.json",
        "meta_clean.json",
        "meta_final.json",
    ]

    if os.path.exists(src_path):
        shutil.move(src_path, dst_path)

    if move_metadata:
        parent_meta_path = os.path.dirname(src_path)

        for filename in os.listdir(parent_meta_path):
            file_path = os.path.join(parent_meta_path, filename)
            if filename in metadata_files:
                shutil.move(file_path, dst_path)

def upload():
    if train_data_path:
        move(train_data_path, tmp_train_data, False)
        zip_file(tmp_dataset)
        upload_dataset(dataset_zip, True)
        os.remove(dataset_zip)
    if logs_path:
        upload_dataset(logs_path, False)

upload()


## 7.3. Upload with GIT (Alternative)

In [ ]:
# @title ### 7.3.1. Clone Repository

clone_model = True  # @param {'type': 'boolean'}
clone_dataset = True  # @param {'type': 'boolean'}

def clone_repository(repo_url, local_path):
    !git lfs install --skip-smudge
    os.environ["GIT_LFS_SKIP_SMUDGE"] = "1"
    !git clone {repo_url} {local_path}

if clone_model:
    clone_repository(f"https://huggingface.co/{model_repo}", f"/content/{model_name}")

if clone_dataset:
    clone_repository(f"https://huggingface.co/datasets/{datasets_repo}", f"/content/{dataset_name}")


In [ ]:
# @title ### 7.3.2. Commit using Git
import os

os.chdir(root_dir)

# @markdown Choose which repo you want to commit
commit_model = True  # @param {'type': 'boolean'}
commit_dataset = True  # @param {'type': 'boolean'}
# @markdown Other Information
commit_message = ""  # @param {type :"string"}

if not commit_message:
    commit_message = f"feat: upload {project_name} lora model and dataset"

!git config --global user.email "example@mail.com"
!git config --global user.name "example"

def commit(repo_folder, commit_message):
    os.chdir(os.path.join(root_dir, repo_folder))
    !git lfs install
    !huggingface-cli lfs-enable-largefiles .
    !git add .
    !git commit -m "{commit_message}"
    !git push


if commit_model:
    commit(model_name, commit_message)

if commit_dataset:
    commit(dataset_name, commit_message)